# Soft K-Means

**NLP Tasks** -- 

1. Read the file and store the list of titles
2. Remove the stopwords and perform stemming
3. Create the word to index
4. Create the input matrix i.e. vocabulary * document data frame
5. Create the tf_idf or the count_vectorizer
6. Run K-Means

In [2]:
# Importing the libraries
import numpy as np
import nltk
import matplotlib.pyplot as plt

In [3]:
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

## Loading the data

In [6]:
titles = [line.rstrip() for line in open("data/books_titles/all_book_titles.txt")]
len(titles)

2373

In [10]:
stopwords = set(w.rstrip() for w in open('data/books_titles/stopwords.txt'))

# Adding few more stopwords
stopwords = stopwords.union({
    'introduction', 'edition', 'series', 'application',
    'approach', 'card', 'access', 'package', 'plus', 'etext',
    'brief', 'vol', 'fundamental', 'guide', 'essential', 'printed',
    'third', 'second', 'fourth', })

In [11]:
# Pre-processing

def my_tokenizer(book_title):
    book_title = book_title.lower()
    tokens = nltk.tokenize.word_tokenize(book_title)
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)]
    return tokens

### Word to index mapping

In [13]:
# create a word-to-index map so that we can create our word-frequency vectors later.


'''
Pseudo-code:
# For each token in the list of tokens
    Check if the token is in the dictionary keys.
    If no, add the token key into the dictionary and corresponding count in the value
    else, increment the counter for that key.
'''

word_index_map = {}  # word: index dictionary
current_index = 0    # Index starts from 0 

# let's also save the tokenized versions so we don't have to tokenize again later.
all_tokens = []      # all the words in each title, includes repeated words

all_titles = []      # Number of documents
index_word_map = []  # List of unique words

for title in titles:
    #print(title)
    #try:
    #title = title.encode('ascii', 'ignore') # this will throw exception if bad characters
    all_titles.append(title)
    tokens = my_tokenizer(title)
    all_tokens.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
            index_word_map.append(token)
#    except:
#        print("Exception")

** Alternate solution **

The idea is to give the lower index values for high frequency words.

Why? Then you could consider only the top 5000 words as dimension instead of using all the words. Since the top 5000 words are 
highly occuring words, it would be safe to take them rather than the previous approach.

Sorting and creating index based on higher values.
Creating word_index_mapping dictionary with the word as the key and value as the index, such that the most frequent word has the 
lower index value.

In [ ]:
'''word_dict = {}

for t in tokens:
    if t in word_dict.keys():
        word_dict[t] += 1
    else: 
        word_dict[t] = 1

word_series = pd.Series(word_dict).sort_values(ascending=False)

word_index = {}
for i,j in enumerate(word_series.index):
    word_index[j] = i

'''    

### Creating the input matrix with the count_values as the cell values

In [9]:
# Now let's create our input matrices - just indicator variables
# Implementing the count_vectorizer

def tokens_to_vector(tokens):
    x = np.zeros(len(word_index_map))   # word_index_map - {word:index}  
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    return x

# NOTE: we are creating an array of D*N dimension i.e. words in the rows and documents in the columns.

N = len(all_titles)      # Number of documents
D = len(word_index_map)  # Number of unique words
X = np.zeros((D, N)) # terms will go along rows, documents along columns
i = 0

for tokens in all_tokens:
    X[:,i] = tokens_to_vector(tokens)   # Filling the title column
    i += 1
    

NameError: name 'all_tokens' is not defined

In [ ]:
def dist_calculation(input_array, centroid):
    
    diff = input_array - centroid
    sqr_dist = np.power(diff, 2)    
    sum_sqr_dist = np.sum(sqr_dist, axis=1)
    
    return sum_sqr_dist 
    #np.exp(-1 * np.sum(np.power((input_array -  centroids[k,:]), 2), axis=1))

In [ ]:
def cost(resp, input_array, centroid):
    cost = 0
# =============================================================================
#     for k in range(len(centroid)):
#         for n in range(len(input_array)):
#             cost += resp[n,k] * dist(centroid[k], input_array[n])
# =============================================================================

    for k in range(len(centroid)):        
        cost += resp[:,k].dot(dist_calculation(input_array, centroid[k]))

    return cost

In [ ]:
def plot_k_means(input_array, 
                 n_clusters, 
                 beta=1,
                 max_iter=20, 
                 show_plots=False):
    
    nrow = len(input_array)
    
    np.random.seed(100)
    init_centroid_index = np.random.randint(0, nrow, n_clusters)
    centroids = input_array[init_centroid_index]

    exponents = np.empty((nrow, n_clusters))
    costs = np.zeros(max_iter)
    
    last_iter_num = 0
    
    for iter_num in range(max_iter):        
     
        # Step 1: Calculating the responsibility
        
# =============================================================================
#         for k in range(n_clusters):
#             for n in range(nrow):
#                 exponents[n,k] = np.exp(-beta * dist(input_array[n,:], centroids[k,:]))
# =============================================================================
    
        # Vectorization       
        for k in range(n_clusters):          
            exponents[:, k] = np.exp(-1 * dist_calculation(input_array, centroids[k,:]))
            #exponents[:, k] = np.exp(-beta * np.sum(np.power((input_array - centroids[k,:]), 2), axis=1))         
            
        resp = exponents / np.sum(exponents, axis=1, keepdims=True)
        
        # Step 2: Recalculate the means
        for k in range(n_clusters):
            centroids[k] = resp[:, k].dot(input_array) / np.sum(resp[:, k])
        
        costs[iter_num] = cost(resp, input_array, centroids)
        
        if iter_num > 0:
            if np.abs(costs[iter_num] - costs[iter_num -1]) < 10e-5:
                print("converged at iteration number {0:d}".format(iter_num))
                last_iter_num = iter_num
                break;
    
    #cluster_assignment = resp.argmax(axis=1)
        
    if show_plots:
        print(costs)
        plt.plot(costs[0:last_iter_num])
        plt.title("Costs")
        plt.show()

        random_colors = np.random.random((n_clusters, 3))
        colors = resp.dot(random_colors)    
        plt.scatter(input_array[:,0], input_array[:,1], c=colors)
        plt.show()

    return centroids, resp

In [1]:
centroids, resp = plot_k_means(X, n_clusters=10, 
                 beta=1,
                 max_iter=20, 
                 show_plots=False)

NameError: name 'all_tokens' is not defined

In [ ]:
cluster_assignment = resp.argmax(axis=1)

len(cluster_assignment) 